In [1]:
import pandas as pd
import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin
# import featuretools as ft
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
# matplotlib and seaborn for plotting
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
import warnings
warnings.filterwarnings("ignore")

In [3]:
application_train = pd.read_csv('../data/application_train.csv')
application_test = pd.read_csv('../data/application_test.csv')
bureau = pd.read_csv('../data/bureau.csv')
bureau_balance = pd.read_csv('../data/bureau_balance.csv')
credit_card_balance = pd.read_csv('../data/credit_card_balance.csv')
installments_payments = pd.read_csv('../data/installments_payments.csv')
pos_cash_balance = pd.read_csv('../data/POS_CASH_balance.csv')
previous_application = pd.read_csv('../data/previous_application.csv')


homecredit_columns_description = pd.read_csv('../data/HomeCredit_columns_description.csv',encoding='latin1')

## 1. Application_train.csv

### Feature Creation: Decision Tree Discretization for external features

In [7]:
def decisionTreeDiscretization(df_,target_, feature):
    df = df_.copy()
    df = df.fillna(0)
    target = target_.copy()
    score_ls = []     # store the roc auc
    score_pred_ls = [] # store the predictions
    for tree_depth in range(1,11):
        tree_model = DecisionTreeClassifier(max_depth=tree_depth)
        scores = cross_val_score(tree_model, df[feature].to_frame(),target_.to_frame(),cv=3, scoring='roc_auc')
        score_ls.append(np.mean(scores))
        tree_model.fit(df[feature].to_frame(), target)
        score_pred_ls.append(tree_model.predict_proba(df[feature].to_frame())[:,1])
    final = list(score_pred_ls[score_ls.index(max(score_ls))])
    mapping = {v:k for k,v in enumerate(set(final))}
    output = [mapping[y] for y in final]
    return pd.DataFrame({feature + "_DISCRETISED":output,"SK_ID_CURR": df['SK_ID_CURR']}).set_index("SK_ID_CURR")

def discretize(df,target):
    discretization_features = ['EXT_SOURCE_2','EXT_SOURCE_3','EXT_SOURCE_1']
    features = decisionTreeDiscretization(df,
                                          target,
                                          discretization_features.pop())
    while discretization_features:
        df_ = decisionTreeDiscretization(df,
                                        target,
                                        discretization_features.pop()) 
        features = features.join(df_,on='SK_ID_CURR')
    return features

In [8]:
application_train_features = discretize(application_train,application_train['TARGET'])

In [9]:
application_train_features.head(5)

,EXT_SOURCE_1_DISCRETISED,EXT_SOURCE_3_DISCRETISED,EXT_SOURCE_2_DISCRETISED
SK_ID_CURR,,,
100002,0,8,0
100003,10,15,12
100004,8,9,1
100006,8,15,12
100007,8,15,0


# 2. Bureau Dataset

a. parent dataset: bureau.csv 
   - count aggregation features created
   - average aggregation features created
   - debt:credit ratio feature created
   
b. child dataset: bureau_balance.csv
   - rolling window credit loan status feature will be created and joined to parent dataset
   
c. combinig/joining both datasets, which will be aggregated in line with primary key ("SK_ID_CURR) of application_train (target dataframe) with the following features:
   - count aggregation features created
   - average aggregation features created
   - debt:credit ratio feature created 
   - rolling window credit loan status feature will be created and joined to parent dataset

## a. Child Dataset: bureau_balance.csv

### Feature Creation: Rolling Window Credit Loan Status

rolling window Exponential Moving Average is derived and the mean is used as the feature

In [10]:
def bureauBalanceRollingCreditLoan(df):
    df_final = df.copy()
    df_final['STATUS'] = df_final['STATUS'].replace(['X','C'],'0')
    df_final['STATUS'] = pd.to_numeric(df_final['STATUS'])
    df_final = df_final.groupby("SK_ID_BUREAU")['STATUS'].agg(
        lambda x: x.ewm(span=x.shape[0], adjust=False).mean().mean()
    )
    df_final = df_final.reset_index(name="CREDIT_STATUS_EMA_AVG")
    df_final = df_final.set_index('SK_ID_BUREAU')
    return df_final

In [11]:
bureauBalanceRollingCreditLoan(bureau_balance)

,CREDIT_STATUS_EMA_AVG
SK_ID_BUREAU,
5001709,0.000000
5001710,0.000000
5001711,0.000000
5001712,0.000000
5001713,0.000000
...,...
6842884,0.000000
6842885,4.014966
6842886,0.000000


## b. Parent Dataset: bureau.csv

### Feature Creation: Aggregation Features - Count 

- Number of loans
- Number of loans prolonged
- Percentage of active loans
- Number of type of loans



In [8]:
def aggCountBureau(df):
    agg = df.groupby("SK_ID_CURR")
    # count number of loans
    df_final = pd.DataFrame(agg['SK_ID_CURR'].agg('count').reset_index(name='NUM_CREDIT_COUNT'))
    # count number of loans prolonged
    loans_prolonged = agg['CNT_CREDIT_PROLONG'].sum().reset_index(name='CREDIT_PROLONG_COUNT').set_index("SK_ID_CURR")
    df_final = df_final.join(loans_prolonged,on='SK_ID_CURR')
    # count percentage of active loans
    active_loans = agg['CREDIT_ACTIVE'].value_counts().reset_index(name='ACTIVE_LOANS_COUNT')
    active_loans = active_loans[active_loans['CREDIT_ACTIVE'] == 'Active'][['SK_ID_CURR','ACTIVE_LOANS_COUNT']].set_index("SK_ID_CURR")
    df_final = df_final.join(active_loans,on='SK_ID_CURR')
    df_final['ACTIVE_LOANS_PERCENT'] = df_final['ACTIVE_LOANS_COUNT']/df_final['NUM_CREDIT_COUNT']
    df_final.drop(["ACTIVE_LOANS_COUNT"], axis=1, inplace=True)
    df_final['ACTIVE_LOANS_PERCENT'] = df_final['ACTIVE_LOANS_PERCENT'].fillna(0)
    # count credit type
    # one hot encode
    ohe = OneHotEncoder(sparse=False)
    ohe_fit = ohe.fit_transform(df[["CREDIT_TYPE"]])
    credit_type = pd.DataFrame(ohe_fit, columns = ohe.get_feature_names(["CREDIT_TYPE"]))
    credit_type.insert(loc=0, column='SK_ID_CURR', value=df['SK_ID_CURR'])
    credit_type = credit_type.groupby("SK_ID_CURR").sum()
    df_final = df_final.join(credit_type, on="SK_ID_CURR")
    df_final = df_final.set_index("SK_ID_CURR")
    return df_final

In [9]:
aggCountBureau(bureau)

,NUM_CREDIT_COUNT,CREDIT_PROLONG_COUNT,ACTIVE_LOANS_PERCENT,CREDIT_TYPE_Another type of loan,CREDIT_TYPE_Car loan,CREDIT_TYPE_Cash loan (non-earmarked),CREDIT_TYPE_Consumer credit,CREDIT_TYPE_Credit card,CREDIT_TYPE_Interbank credit,CREDIT_TYPE_Loan for business development,CREDIT_TYPE_Loan for purchase of shares (margin lending),CREDIT_TYPE_Loan for the purchase of equipment,CREDIT_TYPE_Loan for working capital replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mobile operator loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan
SK_ID_CURR,,,,,,,,,,,,,,,,,,
100001,7,0,0.428571,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100002,8,0,0.250000,0.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100003,4,0,0.250000,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100004,2,0,0.000000,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100005,3,0,0.666667,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456249,13,0,0.153846,1.0,0.0,0.0,9.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
456250,3,0,0.666667,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
456253,4,0,0.500000,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Feature Creation: Aggregation Features - Average

- Average number of days between loans
- Average number of overdue days of overdue loans

In [10]:
def aggAvgBureau(df):
    agg = df.groupby('SK_ID_CURR')
    # average of CREDIT_DAY_OVERDUE
    final_df = agg['CREDIT_DAY_OVERDUE'].mean().reset_index(name = "CREDIT_DAY_OVERDUE_MEAN")
    # average of days between credits of DAYS_CREDIT
    days_credit_between = pd.DataFrame(df['SK_ID_CURR'])
    days_credit_between['diff'] = agg['DAYS_CREDIT'].diff()
    days_credit_between = days_credit_between.groupby("SK_ID_CURR")['diff'].mean().reset_index(name = 'DAYS_CREDIT_BETWEEN_MEAN')
    days_credit_between.set_index("SK_ID_CURR",inplace=True)
    final_df = final_df.join(days_credit_between, on='SK_ID_CURR')
    final_df = final_df.set_index("SK_ID_CURR")
    return final_df

In [11]:
aggAvgBureau(bureau)

,CREDIT_DAY_OVERDUE_MEAN,DAYS_CREDIT_BETWEEN_MEAN
SK_ID_CURR,,
100001,0.0,89.500000
100002,0.0,11.714286
100003,0.0,660.000000
100004,0.0,918.000000
100005,0.0,155.500000
...,...,...
456249,0.0,45.916667
456250,0.0,89.000000
456253,0.0,68.666667


### Feature Creation: debt credit ratio

ratio of AMT_CREDIT_SUM_DEBT to AMT_CREDIT_SUM created

In [12]:
def debtCreditRatio(df):
    #get debt:credit ratio
    df['DEBT_CREDIT_RATIO'] = df['AMT_CREDIT_SUM_DEBT']/df['AMT_CREDIT_SUM']
    df_final = df.groupby('SK_ID_CURR')['DEBT_CREDIT_RATIO'].mean().reset_index(name='DEBT_CREDIT_RATIO')
    df_final = df_final.set_index("SK_ID_CURR")   
    return df_final

In [13]:
debtCreditRatio(bureau)

,DEBT_CREDIT_RATIO
SK_ID_CURR,
100001,0.282518
100002,0.136545
100003,0.000000
100004,0.000000
100005,0.601256
...,...
456249,0.090595
456250,0.575079
456253,0.299308


## C. Combining bureau.csv and bureau_balance.csv

In [14]:
def bureauFeatures(bureau, bureau_balance):
    dfs = []
    # handling features for bureau_balance
    bureau_balance_rolling_loan = bureauBalanceRollingCreditLoan(bureau_balance)
    bureau_df = bureau.copy()
    bureau_df = bureau_df.join(bureau_balance_rolling_loan,on="SK_ID_BUREAU")
    bureau_df["CREDIT_STATUS_EMA_AVG"] = bureau_df['CREDIT_STATUS_EMA_AVG'].fillna(0)
    bureau_df = bureau_df.groupby("SK_ID_CURR")["CREDIT_STATUS_EMA_AVG"].mean()
    dfs.append(bureau_df)
    dfs.append(aggCountBureau(bureau))
    dfs.append(aggAvgBureau(bureau))
    dfs.append(debtCreditRatio(bureau))
    final_df = dfs.pop()
    while dfs:
        final_df = final_df.join(dfs.pop(),on='SK_ID_CURR')
    return final_df

In [15]:
bureau_features = bureauFeatures(bureau, bureau_balance)
bureau_features

,DEBT_CREDIT_RATIO,CREDIT_DAY_OVERDUE_MEAN,DAYS_CREDIT_BETWEEN_MEAN,NUM_CREDIT_COUNT,CREDIT_PROLONG_COUNT,ACTIVE_LOANS_PERCENT,CREDIT_TYPE_Another type of loan,CREDIT_TYPE_Car loan,CREDIT_TYPE_Cash loan (non-earmarked),CREDIT_TYPE_Consumer credit,...,CREDIT_TYPE_Loan for business development,CREDIT_TYPE_Loan for purchase of shares (margin lending),CREDIT_TYPE_Loan for the purchase of equipment,CREDIT_TYPE_Loan for working capital replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mobile operator loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan,CREDIT_STATUS_EMA_AVG
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,0.282518,0.0,89.500000,7,0,0.428571,0.0,0.0,0.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.065031
100002,0.136545,0.0,11.714286,8,0,0.250000,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.183594
100003,0.000000,0.0,660.000000,4,0,0.250000,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
100004,0.000000,0.0,918.000000,2,0,0.000000,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
100005,0.601256,0.0,155.500000,3,0,0.666667,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456249,0.090595,0.0,45.916667,13,0,0.153846,1.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
456250,0.575079,0.0,89.000000,3,0,0.666667,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
456253,0.299308,0.0,68.666667,4,0,0.500000,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


## 3. Installments_payments.csv

The following features are created and used from installments_payments.csv:
- percentage of missed installments payments
- Average percentage of unpaid payments for each missed payment
- Average of unpaid payments for each missed payment

### Feature Creation: Installment Payments Features

In [16]:
def aggAvgInstalments(df):
    df_ = df.copy()
    df_['INSTALMENT_MISSED'] = (df_['AMT_INSTALMENT'] > df_['AMT_PAYMENT']).astype(int)
    df_['AMT_UNPAID'] = df_['AMT_INSTALMENT'] - df_['AMT_PAYMENT']
    df_['PERC_UNPAID'] = df_['AMT_UNPAID']/df_['AMT_INSTALMENT']
    df_ = df_.fillna(0)
    agg = df_.groupby("SK_ID_CURR")
    # percentage of missed payments
    missed_instalments = agg['INSTALMENT_MISSED'].agg(lambda x: x.sum()/x.count()). \
        reset_index().set_index("SK_ID_CURR")
    # percentage of payments difference for each missed payment
    avg_percent_unpaid = agg['PERC_UNPAID'].mean().reset_index().set_index("SK_ID_CURR")
    # average payments difference for each missed payment
    avg_unpaid = agg['AMT_UNPAID'].mean().reset_index().set_index("SK_ID_CURR")
    final_df = missed_instalments
    final_df = final_df.join(avg_percent_unpaid, on='SK_ID_CURR')
    final_df = final_df.join(avg_unpaid,on="SK_ID_CURR")
    return final_df

In [17]:
installment_payments_features = aggAvgInstalments(installments_payments)
installment_payments_features

,INSTALMENT_MISSED,PERC_UNPAID,AMT_UNPAID
SK_ID_CURR,,,
100001,0.000000,0.000000,0.00000
100002,0.000000,0.000000,0.00000
100003,0.000000,0.000000,0.00000
100004,0.000000,0.000000,0.00000
100005,0.000000,0.000000,0.00000
...,...,...,...
456251,0.000000,0.000000,0.00000
456252,0.000000,0.000000,0.00000
456253,0.142857,0.071429,283.79250


## 4. Credit_Card_Balance.csv

The following features are created and used from credit_card_balance.csv
- Average credit balance
- rolling window credit balance mean

### Feature Creation: Average Credit Balance

In [18]:
def avgCreditBalance(df):
    return df.groupby('SK_ID_CURR')['AMT_BALANCE'].mean()

In [19]:
avgCreditBalance(credit_card_balance)

SK_ID_CURR
100006         0.000000
100011     54482.111149
100013     18159.919219
100021         0.000000
100023         0.000000
              ...      
456244    131834.730732
456246     13136.731875
456247     23216.396211
456248         0.000000
456250    173589.326250
Name: AMT_BALANCE, Length: 103558, dtype: float64

### Feature Creation: Rolling Window EMA Credit Balance Mean

In [20]:
def creditCardBalanceRollingBalance(df):
    df_final = df.copy()
    df_final = df_final.sort_values(by="MONTHS_BALANCE")
    df_final = df_final.groupby("SK_ID_CURR")['AMT_BALANCE'].agg(
        lambda x: x.ewm(span=x.shape[0], adjust=False).mean().mean()
    )
    df_final = df_final.reset_index(name="CREDIT_CARD_BALANCE_EMA_AVG")
    df_final = df_final.set_index('SK_ID_CURR')
    return df_final

In [21]:
creditCardBalanceRollingBalance(credit_card_balance)

,CREDIT_CARD_BALANCE_EMA_AVG
SK_ID_CURR,
100006,0.000000
100011,124432.830831
100013,14663.956505
100021,0.000000
100023,0.000000
...,...
456244,235885.879933
456246,20346.726563
456247,59190.301167


### Combining Features for Credit Card Balance

In [22]:
def creditCardFeatures(credit_card_balance):
    dfs = []
    dfs.append(avgCreditBalance(credit_card_balance))
    dfs.append(creditCardBalanceRollingBalance(credit_card_balance))
    final_df = dfs.pop()
    while dfs:
        final_df = final_df.join(dfs.pop(),on='SK_ID_CURR')
    return final_df

In [23]:
credit_card_balance_features = creditCardFeatures(credit_card_balance)
credit_card_balance_features

,CREDIT_CARD_BALANCE_EMA_AVG,AMT_BALANCE
SK_ID_CURR,,
100006,0.000000,0.000000
100011,124432.830831,54482.111149
100013,14663.956505,18159.919219
100021,0.000000,0.000000
100023,0.000000,0.000000
...,...,...
456244,235885.879933,131834.730732
456246,20346.726563,13136.731875
456247,59190.301167,23216.396211


# Combining All Features into application_train.csv

In [24]:
final_features = application_train_features.join(bureau_features,on="SK_ID_CURR")
final_features = final_features.join(installment_payments_features,on="SK_ID_CURR")
final_features = final_features.join(credit_card_balance_features,on="SK_ID_CURR")
final_features = final_features.fillna(0)
final_features

,EXT_SOURCE_1_DISCRETISED,EXT_SOURCE_3_DISCRETISED,EXT_SOURCE_2_DISCRETISED,DEBT_CREDIT_RATIO,CREDIT_DAY_OVERDUE_MEAN,DAYS_CREDIT_BETWEEN_MEAN,NUM_CREDIT_COUNT,CREDIT_PROLONG_COUNT,ACTIVE_LOANS_PERCENT,CREDIT_TYPE_Another type of loan,...,CREDIT_TYPE_Mobile operator loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan,CREDIT_STATUS_EMA_AVG,INSTALMENT_MISSED,PERC_UNPAID,AMT_UNPAID,CREDIT_CARD_BALANCE_EMA_AVG,AMT_BALANCE
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,0,8,0,0.136545,0.0,11.714286,8.0,0.0,0.250000,0.0,...,0.0,0.0,0.0,0.0,0.183594,0.000000,0.000000,0.000000,0.0,0.0
100003,10,15,12,0.000000,0.0,660.000000,4.0,0.0,0.250000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
100004,8,9,1,0.000000,0.0,918.000000,2.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
100006,8,15,12,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
100007,8,15,0,0.000000,0.0,0.000000,1.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.090909,0.045455,452.384318,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251,1,15,13,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
456252,8,15,4,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
456253,13,12,6,0.299308,0.0,68.666667,4.0,0.0,0.500000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.142857,0.071429,283.792500,0.0,0.0


# Feature Views

## Tables:
- application_train.csv
- bureau.csv
- bureau_balance.csv
- previous_application.csv
- POS_CASH_balance.csv
- credit_card_balance.csv
- installments_payments.csv



![Tables](https://storage.googleapis.com/kaggle-media/competitions/home-credit/home_credit.png)

### Entity: 
-  `SK_ID_CURR`



### Feature Views:
-  `static_feature_view`: customers demographic features, age, income, occupation, indicators and customer scores.


 'NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'OCCUPATION_TYPE',
 'ORGANIZATION_TYPE',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 ....
 


- `bureau_feature_view`: customer's credit info with other finanacial instuitues reported to Credit Bureau

 CREDIT_STATUS_EMA_AVG NUM_CREDIT_COUNT	CREDIT_PROLONG_COUNT  ACTIVE_LOANS_PERCENT ...  CREDIT_DAY_OVERDUE_MEAN	DAYS_CREDIT_BETWEEN_MEAN  ....


- `previous_loan_feature_view`: customer's previous credit card, cash loans, and repayment history.

 INSTALMENT_MISSED	PERC_UNPAID	AMT_UNPAID ...
 CREDIT_CARD_BALANCE_EMA_AVG	AMT_BALANCE ...